## Detailed Article Explaination

The detailed code explanation for this article is available at the following link:

https://www.daniweb.com/programming/computer-science/tutorials/541635/7-nlp-tasks-to-perform-for-free-in-python-with-mistral-7b-llm

For my other articles for Daniweb.com, please see this link:

https://www.daniweb.com/members/1235222/usmanmalik57

## Importing and Installing Required Libraries

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip3 install -q -U bitsandbytes==0.42.0
!pip3 install -q -U accelerate==0.27.1

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, logging
from transformers import BitsAndBytesConfig
import torch

## Importing and Configuring the Mistral 7b Model

In [3]:
#Ignore warnings
logging.set_verbosity(logging.CRITICAL)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [4]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
device = "cuda" # the device to load the model onto
model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(model_id)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

## 1. Question Answering with Mistral 7b

In [80]:
def generate_response(input_text, response_tokens, temperature):
  messages = [
      {"role": "user", "content": input_text},
  ]
  encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")

  model_inputs = encodeds.to(device)

  generated_ids = model.generate(model_inputs,
                                max_new_tokens=response_tokens,
                                temperature = temperature,
                                do_sample=True)
  decoded = tokenizer.batch_decode(generated_ids)
  return decoded[0].split("[/INST]")[1].rstrip("</s>")

In [81]:
input_text = "How to bake a pizza?"
response = generate_response(input_text, 1000, 0.1)
print(response)

 Baking a pizza at home can be a fun and delicious experience. Here is a simple recipe to make a classic pizza:

Ingredients:

* Pizza dough
* Tomato sauce
* Mozzarella cheese, shredded
* Toppings of your choice (such as pepperoni, mushrooms, onions, peppers, etc.)
* Olive oil
* Salt and pepper
* Flour or cornmeal

Instructions:

1. Preheat your oven to 450°F (230°C).
2. On a lightly floured surface, roll out the pizza dough to your desired thickness.
3. Brush the dough with a little bit of olive oil and season with salt and pepper.
4. Spread a layer of tomato sauce over the dough, leaving a small border around the edges.
5. Sprinkle a generous amount of shredded mozzarella cheese over the sauce.
6. Add your desired toppings to the pizza.
7. Carefully transfer the pizza to a baking sheet or pizza stone. You may want to sprinkle some cornmeal on the surface to prevent sticking.
8. Bake the pizza for 10-15 minutes, or until the crust is golden brown and the cheese is melted and bubbly.
9

## 2. Text Summarization with Mistral 7b

In [83]:
paragraph = """
A large language model (LLM) is a language model notable for its ability to achieve general-purpose language generation and other natural language processing tasks such as classification.
LLMs acquire these abilities by learning statistical relationships from text documents during a computationally intensive self-supervised and semi-supervised training process.[1]
LLMs can be used for text generation, a form of generative AI, by taking an input text and repeatedly predicting the next token or word.[2] LLMs are artificial neural networks.
The largest and most capable are built with a decoder-only transformer-based architecture while some recent implementations are based on other architectures, such as recurrent neural network variants and Mamba (a state space model)
"""

print(f"Total characters in original paragraph: {len(paragraph)}")
input_text = f"Summarize the following text: {paragraph}"
response = generate_response(input_text, 500, 0.9)
print(f"Total characters in summarized paragraph: {len(response)}")
print(response)

Total characters in original paragraph: 780
Total characters in summarized paragraph: 348
 In summary, a large language model (LLM) is an AI model that can generate and process human-like language. It learns to do this through a computationally intense training process on text documents. LLMs can be used for text generation and can be built using various architectures, including transformer-based and recurrent neural network variants.


## 3. Text Classification with Mistral 7b

In [88]:
review = """
I enjoyed the movie but found it very long at times with boring scenes.
"""

input_text = f"Find the sentiment of this review, your response should only contain  single word 'positive', 'negative', or 'neutral: {review}"
response = generate_response(input_text, 10, 0.1)
print(response)

 neutral


## 4. Text Translation with Mistral 7b

In [90]:
input = """
I am feeling hugry, I think I should go out and have lunch.
"""

input_text = f"Translate the following into French. The response should only contain the French translation: {input}"
response = generate_response(input_text, 100, 0.1)
print(response)

 Je suis fâché, je pense que je devrais sortir et avoir déjeuner.


## 5. Text Generation with Mistral 7b

In [98]:
input_text = "Give me 5 catchy names for an ice cream parlour on a beach"
response = generate_response(input_text, 200, 0.9)
print(response)

 1. "Scoops by the Sea"
2. "Surf and Scoops"
3. "Beach Bites Bakery"
4. "Parlor in Paradise"
5. "Ocean Oasis Ice Cream"


## 6. Code Generation with Mistral 7b

In [92]:
input_text = "Write a Python fuction to add two numbers"
response = generate_response(input_text, 200, 0.1)
print(response)

 Here is a simple Python function that takes two numbers as arguments and returns their sum:

```python
def add_numbers(a, b):
    """Function to add two numbers"""
    return a + b
```

You can use this function like this:

```python
x = 5
y = 3
result = add_numbers(x, y)
print(result)
```
This will output `8`


## 7. Named Entity Recognition

In [101]:
input = """
Ronaldo from Portugal was one of the best players Manchester United ever produced in the Premier League"
"""

input_text = f"Extract name entities from the following text. Response should be in the form word -> entity type: {input}"
response = generate_response(input_text, 100, 0.1)
print(response)

 Ronaldo -> Person
Portugal -> Location
Manchester United -> Organization
Premier League -> Event
